In [1]:
import os
import mmcv

from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [2]:
# config file 들고오기
cfg = Config.fromfile('./configs/_base_/my_seg/upernet_swin_base_wind12_512.py')

epoch = 'epoch_28'

cfg.model.decode_head.num_classes=11
cfg.model.auxiliary_head.num_classes=11

# dataset config 수정
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1

cfg.work_dir = './work_dirs/upper_swin_base'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=cfg.data.samples_per_gpu,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2021-10-22 05:27:54,432 - mmseg - INFO - Loaded 819 images


In [4]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/segmentation/github_workspace/semantic-segmentation-level2-cv-07/mmsegmentation/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '


Use load_from_local loader


In [5]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 8.5 task/s, elapsed: 96s, ETA:     0s

In [22]:
import cv2

In [23]:
# sample_submisson.csv 열기
submission = pd.read_csv('./work_dirs/sample_submission.csv', index_col=None)
json_dir = os.path.join("/opt/ml/segmentation/input/data/test.json")
with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

input_size = 512
output_size = 256
bin_size = input_size // output_size
		
# PredictionString 대입
for image_id, predict in enumerate(output):
    image_id = datas["images"][image_id]
    file_dir = f"/opt/ml/segmentation/input/mmseg/annotations/test/{image_id['id']:04}.png"
    masks = predict.reshape(1, 512, 512)[0]
    cv2.imwrite(file_dir, masks)